In [210]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import *
from pymongo import *

from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
from exp.nb_03 import *

## Define your parameters here
Specify the values for the parameters for lblnew-bestfit-sw.  

In [141]:
param0 = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(30, 250), (300, 250)], ng_refs=[3, 9], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .6, .5), (.55, .55, .55, .65, .65, .7, .8, .9)],
                     cosz=.2588, rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0,
                     atmpro='trp',
                     commitnumber='79f2292')

In [142]:
acs = [('saw', 1), ('trp', 0.2588)]

params = []
for atmpro, cosz in acs:
    dparam = vars(param0)
    dparam['atmpro'] = atmpro
    dparam['cosz'] = cosz
    params.append(LBLnewBestfitSWParam(**dparam))

In [143]:
params

[<class 'exp.nb_00.LBLnewBestfitSWParam'>
 {'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0},
 <class 'exp.nb_00.LBLnewBestfitSWParam'>
 {'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'trp', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 0.2588, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}]

## Run lblnew-bestfit-sw  
Run the Fortran code for lblnew-bestfit-sw in this section.

In [219]:
# Specify the root directory under which you want to run lblnew-bestfit-sw's Fortran code

# Gather most of the runs here:
PATH_FORTRAN = Path('/chia_cluster/home/jackyu/radiation/crdnew-sw/examples')

In [212]:
paths = [PATH_FORTRAN/'/'.join(sorted(f'{n}_{v}' for n, v in vars(p).items())) for p in params]

In [213]:
runners = [LBLnewBestfitSWRun(path=path, param=param) for path, param in zip(paths, params)]

In [214]:
for runner in runners[:1]:
    runner.input_params()
    proc = runner.build()
    out, err = proc.communicate()
    runner.run()

Job <435025> is submitted to queue <serial>.



In [215]:
runners[0].job_id

'435025'

In [152]:
runners[1].job_id

'434166'

In [227]:
! qstat -u jackyu

No matching job found
serial;  type=BATCH;  [ENABLED];  pri=35
2 run;   3 wait;


v4q;  type=BATCH;  [ENABLED];  pri=30
96 run;   0 wait;


v4qexp;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


LCCR_Q;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


URBAN_Q;  type=BATCH;  [ENABLED];  pri=30
576 run;   0 wait;


orc;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


RCEC_Q;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


medium_priority;  type=BATCH;  [ENABLED];  pri=10
0 run;   0 wait;




In [230]:
runners[0].param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

## Run lblnew-bestfit-sw analysis  
Run the analysis notebook for lblnew-bestfit-sw in this section.

In [220]:
# Specify the root directory under which you want to run analysis of lblnew-bestfit-sw's output data

# Gather most of the analysis notebooks here:
PATH_ANALYSIS = Path('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/')

In [226]:
paths = [PATH_ANALYSIS/'/'.join(sorted(f'{n}_{v}' for n, v in vars(r.param).items())) for r in runners]
analyses = [LBLnewBestfitSWAnalysis(path=r.path, runner=r) for r, path in zip(runners, paths)]

In [161]:
for analysis in analyses:
    analysis.run()
    proc = analysis.gitcommit()
    out, err = proc.communicate()
    print('Git-commit\n', f'stdout: {out.decode()}\n', f'stderr: {err.decode()}', '\n')

Git-add  
Git-commit
 stdout: [master 7c2d462] band09 h2o saw cosz=1 nf_refs=[3, 9]
 1 file changed, 1841 insertions(+)
 create mode 100644 atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb

 stderr:  

Git-add  
Git-commit
 stdout: [master 81acc78] band09 h2o trp cosz=0.2588 nf_refs=[3, 9]
 1 file changed, 1841 insertions(+)
 create mode 100644 atmpro_trp/band_9/commitnumber_79f2292/conc_atmpro/cosz_0.2588/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb

 stderr:  



In [205]:
for analysis in analyses: 
    name_gitrepo = PRE_URL.parents[1].stem
    print(str(PRE_URL) + str(analysis.path).split(name_gitrepo)[-1] + '/analysis_-_lblnew-bestfit-sw.ipynb')
    print()

https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb

https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_trp/band_9/commitnumber_79f2292/conc_atmpro/cosz_0.2588/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb



# fin